In [129]:
import pywt
import os
from pylab import *
from numpy import*
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

#### Обработка графиков ЭКГ и преобразование в таблицу

In [2]:
def get_ecg(key):
  way = 'train/' + key + '.npy'
  ecgsig = np.load(way).transpose().reshape((60000,))
  x = np.linspace(0,  1,  num = 2048)
  data = np.sin(250 * np.pi * x**2)
  coefs = pywt.downcoef('a', data, 'db20', mode='symmetric', level=1)
  st='sym5'
  coeffs = pywt.wavedec(ecgsig, st, level=5)
  row = coeffs[0]
  return row

In [3]:
def into_one_table(keys):
  row = get_ecg(keys[0])
  row = np.insert(row, 0, int(keys[0].strip('_hr')))
  table = [row]
  for i in range(target.index.start + 1, target.index.stop):
    row = get_ecg(keys[i])
    row = np.insert(row, 0, int(keys[i].strip('_hr')))
    table = np.vstack([table, row])
  table = pd.DataFrame(table)
  return table

In [4]:
target = pd.Series(list(map(lambda x: x[:-4], os.listdir('train')[:-2])))
ECG_data = into_one_table(target)

In [5]:
ECG_data[0] = ECG_data[0].astype('int')
ECG_data = ECG_data.rename(columns={0: 'ecg_id'})

In [6]:
ECG_data = ECG_data.sort_values(by=['ecg_id'], ascending=True)

In [7]:
ECG_data

,ecg_id,1,2,3,4,5,6,7,8,9,...,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883
0,9,-0.170442,-0.066933,-0.140079,-0.478502,-1.245167,-0.979738,-0.738943,-0.476043,-0.349288,...,0.558936,0.551860,0.551007,0.561618,0.556675,0.645774,0.744957,0.649421,0.667106,0.731759
1,34,0.399648,0.579593,0.460080,-0.077166,-1.270282,-1.040847,-1.689153,-2.481032,-2.205554,...,1.113606,1.088111,1.099680,1.132341,1.130250,1.788834,2.455783,1.818630,1.938019,2.363243
2,43,0.411589,0.365335,0.399006,0.554728,0.911140,0.774487,0.576258,0.404740,0.229235,...,0.341701,0.340029,0.342162,0.341734,0.340499,0.346990,0.366654,0.343959,0.348800,0.365320
3,52,-0.162251,-0.159025,-0.159327,-0.163111,-0.158814,-0.159398,-0.162262,-0.174764,-0.179583,...,-0.267047,-0.258175,-0.261318,-0.274144,-0.273755,-0.519687,-0.765086,-0.532849,-0.575938,-0.730437
4,57,-0.268330,-0.279812,-0.270770,-0.232069,-0.136626,-0.155710,-0.117283,-0.096148,-0.074360,...,-0.304742,-0.297821,-0.302716,-0.310384,-0.311176,-0.513279,-0.714933,-0.521431,-0.558040,-0.686983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,21784,-0.094946,-0.097736,-0.095932,-0.087848,-0.070772,-0.076037,-0.082161,-0.089443,-0.092388,...,0.005842,0.004748,0.005003,0.006885,0.006829,0.041791,0.079657,0.044349,0.050701,0.074361
2097,21795,-0.194416,-0.182553,-0.191956,-0.238293,-0.352612,-0.314071,-0.319038,-0.313175,-0.285890,...,0.120157,0.117738,0.121368,0.121892,0.122540,0.189928,0.260724,0.190226,0.204078,0.251573
2098,21825,-0.004072,-0.000198,-0.003268,-0.017719,-0.054790,-0.048271,-0.088630,-0.084493,-0.070101,...,-0.128999,-0.124998,-0.127924,-0.131497,-0.131004,-0.222249,-0.318699,-0.224482,-0.242503,-0.305902
2099,21831,-0.128454,-0.127641,-0.128944,-0.136319,-0.158699,-0.147968,-0.147763,-0.142817,-0.138197,...,-0.159457,-0.155589,-0.158779,-0.161389,-0.160510,-0.233738,-0.313892,-0.233884,-0.249503,-0.303710


## N-граммы

In [8]:
import neurokit2
import sleepecg
import wfdb.processing

In [9]:
def find_peaks(ecgsignal, peak):
    return wfdb.processing.correct_peaks(ecgsignal, rpeaks,
            search_radius=36, smooth_window_size=50, peak_dir=peak)

In [10]:
def get_cycle(ecgsignal, rpeaks): # разбиение ЭКГ на циклы
  ecgsignal_split = []
  cycle = []
  rpeaks = rpeaks[::-1]
  f = True #флаг конца массива с пиками
  f1 = True #флаг вхождения первого пика
  for i in range(len(ecgsignal)):
    if f and i == rpeaks[-1]:
      if f1:
        f1 = False
        cycle = []
      else:
        ecgsignal_split.append(cycle)
        cycle = []
      rpeaks = rpeaks[:-1]
      if f and len(rpeaks) == 0:
        f = False
    cycle.append(ecgsignal[i])
  return ecgsignal_split

In [11]:
def sign(x):
   return 1 if x > 0 else -1

In [12]:
def RTL(ecgsignal_split):
    sp = [[(i[0] - min(i[:10])), len(i), ((i[0] - min(i[:10]))/len(i))] for i in ecgsignal_split]
    s = []
    for i in range(len(sp) - 1):
        R = sp[i + 1][0] - sp[i][0]
        T = sp[i + 1][1] - sp[i][1]
        L = sp[i + 1][2] - sp[i][2]
        s.append([R, T, L])
    return s

In [13]:
def letters(data): # кодирование цикла по В.М. Успенскому 
  if data[0] == 1:
    if data[1] == -1:
      return 'C'
    else:
      if data[2] == -1:
        return 'E'
      return 'A'
  else:
    if data[1] == 1:
      return 'D'
    else:
      if data[2] == 1:
        return 'B'
      return 'F'

In [14]:
def encoding(ecgsignal_split):
    s = [[sign(i) for i in k] for k in RTL(ecgsignal_split)]
    stroka = ''
    for i in s:
        stroka += letters(i)
    return stroka

In [15]:
ECG_data_array = np.array(ECG_data)

In [16]:
ecg_ids = []
encoding_ecgs = []

In [17]:
for ecgsignal in ECG_data_array:
    ecg_id = ecgsignal[0]
    ecgsignal = ecgsignal[1:]
    
    _, results = neurokit2.ecg_peaks(ecgsignal, sampling_rate=100)
    rpeaks = results["ECG_R_Peaks"]
    rpeaks = sleepecg.detect_heartbeats(ecgsignal, fs=100)
    rpeaks = wfdb.processing.xqrs_detect(ecgsignal, fs=100, verbose=False)
    
    rpeaks = find_peaks(ecgsignal, 'up')
    
    ecgsignal_split = get_cycle(ecgsignal, rpeaks)
    encoding_ecg = encoding(ecgsignal_split)
    
    ecg_ids.append(ecg_id)
    encoding_ecgs.append(encoding_ecg)

In [18]:
encoding_ecg = pd.Series(encoding_ecgs, index=ecg_ids)

In [19]:
encoding_ecg

9.0                   DDCADDCC
34.0       DDDCCCFFFDDFCCCACFE
43.0               BFABCFADDEC
52.0               DDEFBCCBCCF
57.0                 DDCADBCDD
                  ...         
21784.0         DDCCFFAFAFFAFC
21795.0                       
21825.0              ECCDDDECB
21831.0           DAFCCDDDCFCD
21834.0              CDFCCDAFF
Length: 2101, dtype: object

In [20]:
gts = pd.read_csv('train/train_gts.csv')
gts.record_name = gts.record_name.map(lambda x: float(x[:-3]))
gts = gts.sort_values(by='record_name')
gts.insert(2, 'encoding_ecg', array(encoding_ecg))
gts

,record_name,myocard,encoding_ecg
265,9.0,0,DDCADDCC
1407,34.0,0,DDDCCCFFFDDFCCCACFE
1833,43.0,0,BFABCFADDEC
441,52.0,0,DDEFBCCBCCF
1834,57.0,0,DDCADBCDD
...,...,...,...
1753,21784.0,0,DDCCFFAFAFFAFC
2008,21795.0,0,
72,21825.0,0,ECCDDDECB
2100,21831.0,0,DAFCCDDDCFCD


In [21]:
sick = gts[gts.myocard == 1].encoding_ecg
healthy = gts[gts.myocard == 0].encoding_ecg

In [23]:
from nltk.util import ngrams

In [99]:
sp_sick= []

for i in sick:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 5))))
    for k in n:
        sp_sick.append(k)

In [100]:
sp_healthy= []

for i in healthy:
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(i), 5))))
    for k in n:
        sp_healthy.append(k)
    

In [101]:
sp_healthy

['DDCAD',
 'DCADD',
 'CADDC',
 'ADDCC',
 'DDDCC',
 'DDCCC',
 'DCCCF',
 'CCCFF',
 'CCFFF',
 'CFFFD',
 'FFFDD',
 'FFDDF',
 'FDDFC',
 'DDFCC',
 'DFCCC',
 'FCCCA',
 'CCCAC',
 'CCACF',
 'CACFE',
 'BFABC',
 'FABCF',
 'ABCFA',
 'BCFAD',
 'CFADD',
 'FADDE',
 'ADDEC',
 'DDEFB',
 'DEFBC',
 'EFBCC',
 'FBCCB',
 'BCCBC',
 'CCBCC',
 'CBCCF',
 'DDCAD',
 'DCADB',
 'CADBC',
 'ADBCD',
 'DBCDD',
 'DDBBC',
 'DBBCC',
 'BBCCD',
 'BCCDD',
 'CCDDA',
 'CAFCB',
 'AFCBA',
 'FCBAD',
 'CBADD',
 'BADDC',
 'FFCCF',
 'FCCFD',
 'CCFDC',
 'CFDCF',
 'FDCFD',
 'DCFDA',
 'CCDDD',
 'CDDDE',
 'DDDEC',
 'DDECF',
 'DECFF',
 'ECFFC',
 'CFFCC',
 'FFCCF',
 'FCCFF',
 'CCFFC',
 'CFFCC',
 'FFCCC',
 'FCCCD',
 'CCCDC',
 'BEBCE',
 'EBCED',
 'BCEDB',
 'CEDBD',
 'EDBDE',
 'DBDEC',
 'BDDCE',
 'DDCED',
 'DCEDF',
 'CEDFC',
 'EDFCD',
 'DFCDE',
 'DCCDD',
 'CCDDC',
 'CDDCD',
 'DDCDD',
 'DCDDC',
 'CDDCF',
 'DCDAB',
 'CDABD',
 'DABDC',
 'ABDCD',
 'BDCDA',
 'DCDAA',
 'CDAAD',
 'DAADD',
 'AADDB',
 'DDCFC',
 'DCFCC',
 'CFCCF',
 'FCCFD',
 'CCFDD',


In [102]:
def counter_and_sort(arr):
  quantity = {}
  for i in arr:
    if i in quantity:
      quantity[i] += 1
    else:
      quantity[i] = 1
  quantity = pd.Series(list(quantity.values()), index=list(quantity.keys())).sort_values(ascending=False)
  return quantity



In [103]:
q_sick = counter_and_sort(sp_sick)
q_health = counter_and_sort(sp_healthy)

In [107]:
only_sick = list(set(q_sick.index) - set(q_health.index))
only_health = list(set(q_health.index) - set(q_sick.index))

In [131]:
popular_sick = q_sick.iloc[:len(q_sick)//2]
popular_health = q_health.iloc[:len(q_health)//2]

In [135]:
popular_sick_only = list(set(popular_sick.index) - set(popular_health.index))
popular_health_only = list(set(popular_health.index) - set(popular_sick.index))

In [110]:
array(gts)

array([[9.0, 0, 'DDCADDCC'],
       [34.0, 0, 'DDDCCCFFFDDFCCCACFE'],
       [43.0, 0, 'BFABCFADDEC'],
       ...,
       [21825.0, 0, 'ECCDDDECB'],
       [21831.0, 0, 'DAFCCDDDCFCD'],
       [21834.0, 1, 'CDFCCDAFF']], dtype=object)

In [120]:
ngramms_ecgs = []
for ecg in array(gts):
    ngramms_ecg = set()
    n = list(map(lambda x: ''.join(list(x)), list(ngrams(list(ecg[2]), 5))))
    for k in n:
        ngramms_ecg.add(k)
    ngramms_ecgs.append(list(ngramms_ecg))

In [178]:
def result(ng):
    ng_health = sum(list(map(lambda x: 1 if x in only_health else 0, ng)))
    ng_sick = sum(list(map(lambda x: 1 if x in only_sick else 0, ng)))
    
    ng_popular_health = sum(list(map(lambda x: 1 if x in popular_health_only else 0, ng)))
    ng_popular_sick = sum(list(map(lambda x: 1 if x in popular_sick_only else 0, ng)))
    
    res = (ng_health + ng_popular_health + 1)/(ng_sick + ng_popular_sick + 1)
    return 1 if res <= 0.99 else 0

In [179]:
myocard_predict = list(map(lambda x: result(x), ngramms_ecgs))

In [181]:
accuracy_score(gts.myocard, myocard_predict)

0.8724416944312232

In [182]:
recall_score(gts.myocard, myocard_predict) # => многим больным поставили ложный отрицательный диагноз

0.3566265060240964

In [184]:
precision_score(gts.myocard, myocard_predict) # => практически никому из здоровых не поставили ложный положительный диагноз

0.9932885906040269